In [1]:
import numpy as np
import os
import pandas as pd
import re

In [2]:
path = os.getcwd()
print(path)
#/opt/dssg-hot/data/travel_times/pythonscripts

/opt/dssg-hot/data/travel_times/pythonscripts


In [3]:
# get the entry and exit plaza numbers from loop mileposts
entry_exit_plu = pd.read_csv('/opt/dssg-hot/data/milepost_lookups/trips_entry_exit_summary.csv')
entry_exit_plu.head()

,Downloaded HOV & GP?,Illegal? (1 = illegal),Entry Plaza,Exit Plaza,Entry exit concat,Trip def ID 1 (most common),Trip def ID 2 (2nd most common),Missing? (1 = missing),Count,Percentage,Actual entry mile post,Actual exit mile post,Loop entry mile post,Loop exit mile post,Dist btwn entry & exit loop,Actual - loop entry mile post,Actual - loop exit mile post,Note
0,1.0,0,3,3,33,2691,2736.0,1,303676,1.79%,13.74,15.76,13.6,15.63,2.03,0.14,0.13,NaN
1,1.0,0,3,5,35,2691,2736.0,0,502052,2.96%,13.74,19.12,13.6,19.21,5.61,0.14,-0.09,NaN
2,1.0,0,3,6,36,2690,2735.0,1,662877,3.90%,13.74,21.86,13.6,21.83,8.23,0.14,0.03,NaN
3,1.0,0,3,7,37,2690,2735.0,0,225994,1.33%,13.74,20.35,13.6,20.47,6.87,0.14,-0.12,NaN
4,1.0,0,3,9,39,2689,2734.0,1,229669,1.35%,13.74,25.43,13.6,25.17,11.57,0.14,0.25,NaN


In [4]:
# get all xlsx filenames
fpath = '/opt/dssg-hot/data/shirleydata/travel_times_extra/xlsxfiles/'
files = sorted(os.listdir(fpath))
print('# files = ' + str(len(files)))
#print(files)

# files = 182


In [6]:
for file in files:
    
    # get whether it's GP or HOV; lt = lane type
    lt_start_stridx = file.rfind(')')+2
    lt_end_stridx = file.rfind(')')+5
    lt_str = file[lt_start_stridx:lt_end_stridx].strip()
    
    # get the entry and exit mileposts
    entry_start_stridx = file.find('@')+2
    entry_end_stridx = file.find('@')+7
    entry_milepost = float(file[entry_start_stridx:entry_end_stridx])
    exit_start_stridx = file.rfind('@')+2
    exit_end_stridx = file.rfind('@')+7
    exit_milepost = float(file[exit_start_stridx:exit_end_stridx])
    
    # get corresponding entry and exit plazas --> column name
    entry_plaza_float = entry_exit_plu[ \
                (entry_exit_plu['Loop entry mile post'] == entry_milepost) \
                & (entry_exit_plu['Loop exit mile post'] == exit_milepost) \
                                      ]['Entry Plaza']
    exit_plaza_float = entry_exit_plu[ \
                (entry_exit_plu['Loop entry mile post'] == entry_milepost) \
                & (entry_exit_plu['Loop exit mile post'] == exit_milepost) \
                                      ]['Exit Plaza']    
    entry_plaza_str = str(int(entry_plaza_float.values[0])).zfill(2)
    exit_plaza_str = str(int(exit_plaza_float.values[0])).zfill(2)
    colname = lt_str + '_' + entry_plaza_str + '_' + exit_plaza_str 
    
    # make sure everything parsed + looked up correctly
    print(lt_str + ',' + str(entry_milepost) + ',' + str(exit_milepost) + ' --> ' + colname)
    
    # get travel times from current file --> put them into one column
    data = pd.read_excel(fpath+file, 'TravelTimes')
    data.rename(columns={'Unnamed: 0':'Time of day'}, inplace=True)
    melt = pd.melt(data, id_vars=['Time of day'])
    melt['Time']=melt.apply(lambda r : 
           pd.datetime.combine(r['variable'],r['Time of day']),1)
    melt.drop(columns=['Time of day','variable'], inplace=True)
    melt.rename(columns={'value':colname},inplace=True)
    melt.set_index('Time',inplace=True)
    
    # add latest file's column to final df called merged
    if file==files[0]:
        merged = melt
    else:
        merged = pd.merge(merged, melt, how='inner', left_index=True, right_index=True) 

    if file==files[-1]:
        print(merged.head())

GP,28.53,28.98 --> GP_12_12
HOV,28.53,28.98 --> HOV_12_12
GP,27.96,28.53 --> GP_11_11
HOV,27.96,28.53 --> HOV_11_11
GP,27.96,28.98 --> GP_11_12
HOV,27.96,28.98 --> HOV_11_12
GP,27.44,26.16 --> GP_14_14
HOV,27.44,26.16 --> HOV_14_14
GP,27.44,25.17 --> GP_14_15
HOV,27.44,25.17 --> HOV_14_15
GP,27.44,20.93 --> GP_14_18
HOV,27.44,20.93 --> HOV_14_18
GP,27.44,21.83 --> GP_14_16
HOV,27.44,21.83 --> HOV_14_16
GP,27.44,15.08 --> GP_14_21
HOV,27.44,15.08 --> HOV_14_21
GP,27.44,16.12 --> GP_14_20
HOV,27.44,16.12 --> HOV_14_20
GP,27.44,18.7 --> GP_14_17
HOV,27.44,18.7 --> HOV_14_17
GP,27.44,13.92 --> GP_14_22
HOV,27.44,13.92 --> HOV_14_22
GP,26.16,25.17 --> GP_15_15
HOV,26.16,25.17 --> HOV_15_15
GP,26.16,20.93 --> GP_15_18
HOV,26.16,20.93 --> HOV_15_18
GP,26.16,21.83 --> GP_15_16
HOV,26.16,21.83 --> HOV_15_16
GP,26.16,15.08 --> GP_15_21
HOV,26.16,15.08 --> HOV_15_21
GP,26.16,16.12 --> GP_15_20
HOV,26.16,16.12 --> HOV_15_20
GP,26.16,18.7 --> GP_15_17
HOV,26.16,18.7 --> HOV_15_17
GP,26.16,13.92 -->

In [ ]:
merged.to_csv(r'/opt/dssg-hot/data/shirleydata/travel_times_extra/concatenated_405_travel_times_all_times_5min_no_duplicates.csv', index = True, header = True)

In [8]:
# want 5am-8pm only
merged['Time'] = merged.index
merged['hour'] = merged['Time'].apply(lambda t: t.hour)
merged = merged[merged['hour'].between(5, 19, inclusive=True)]
merged.drop('hour', axis=1, inplace=True)
merged.drop('Time', axis=1, inplace=True)
print(merged.head())

                     GP_12_12  HOV_12_12  GP_11_11  HOV_11_11  GP_11_12  \
Time                                                                      
2018-01-01 05:00:00      27.0       27.0      34.2       34.2      61.2   
2018-01-01 05:05:00      27.0       27.0      34.2       34.2      61.2   
2018-01-01 05:10:00      27.0       27.0      34.2       34.2      61.2   
2018-01-01 05:15:00      27.0       27.0      34.2       34.2      61.2   
2018-01-01 05:20:00      27.0       27.0      34.2       34.2      61.2   

                     HOV_11_12  GP_14_14  HOV_14_14  GP_14_15  HOV_14_15  ...  \
Time                                                                      ...   
2018-01-01 05:00:00       61.2      76.8       76.8     136.2      136.2  ...   
2018-01-01 05:05:00       61.2      76.8       76.8     136.2      136.2  ...   
2018-01-01 05:10:00       61.2      76.8       76.8     136.2      136.2  ...   
2018-01-01 05:15:00       61.2      76.8       76.8     136.2      13

In [9]:
# SB10 and SB11 exits are the same loop --> create duplicate columns as follows:
# 13-22 = 13-23, 14-22 = 14-23, 15-22 = 15-23, 16-22 = 16-23, 17-22 = 17-23, 19-22 = 19-23, 20-22 = 20-23, 21-22 = 21-23, 22-22 = 23-23
filter_cols = [col for col in merged if col.endswith('22')]
print(filter_cols)
new_cols = [colname.replace('22', '23') for colname in filter_cols]
print(new_cols)
merged[new_cols] = merged[filter_cols]
merged.head()

['GP_14_22', 'HOV_14_22', 'GP_15_22', 'HOV_15_22', 'GP_13_22', 'HOV_13_22', 'GP_17_22', 'HOV_17_22', 'GP_16_22', 'HOV_16_22', 'GP_22_22', 'HOV_22_22', 'GP_21_22', 'HOV_21_22', 'GP_20_22', 'HOV_20_22', 'GP_19_22', 'HOV_19_22']
['GP_14_23', 'HOV_14_23', 'GP_15_23', 'HOV_15_23', 'GP_13_23', 'HOV_13_23', 'GP_17_23', 'HOV_17_23', 'GP_16_23', 'HOV_16_23', 'GP_23_23', 'HOV_23_23', 'GP_21_23', 'HOV_21_23', 'GP_20_23', 'HOV_20_23', 'GP_19_23', 'HOV_19_23']


,GP_12_12,HOV_12_12,GP_11_11,HOV_11_11,GP_11_12,HOV_11_12,GP_14_14,HOV_14_14,GP_14_15,HOV_14_15,...,GP_16_23,HOV_16_23,GP_23_23,HOV_23_23,GP_21_23,HOV_21_23,GP_20_23,HOV_20_23,GP_19_23,HOV_19_23
Time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 05:00:00,27.0,27.0,34.2,34.2,61.2,61.2,76.8,76.8,136.2,136.2,...,503.4,503.4,102.6,102.6,132.0,132.0,286.8,286.8,393.0,393.0
2018-01-01 05:05:00,27.0,27.0,34.2,34.2,61.2,61.2,76.8,76.8,136.2,136.2,...,503.4,503.4,102.6,102.6,132.0,132.0,286.8,286.8,393.0,393.0
2018-01-01 05:10:00,27.0,27.0,34.2,34.2,61.2,61.2,76.8,76.8,136.2,136.2,...,503.4,503.4,102.6,102.6,132.0,132.0,286.8,286.8,393.0,393.0
2018-01-01 05:15:00,27.0,27.0,34.2,34.2,61.2,61.2,76.8,76.8,136.2,136.2,...,503.4,503.4,102.6,102.6,132.0,132.0,286.8,286.8,393.0,393.0
2018-01-01 05:20:00,27.0,27.0,34.2,34.2,61.2,61.2,76.8,76.8,136.2,136.2,...,503.4,503.4,102.6,102.6,132.0,132.0,286.8,286.8,393.0,393.0


In [10]:
merged.to_csv(r'/opt/dssg-hot/data/travel_times/concatenated_405_travel_times_5am_to_8pm_5min.csv', index = True, header = True)